# Lattice Dynamics and Density Functional Perturbation Theory - Harmonic Level

The script 'thermodyanmics.py' is a comprehensive module for lattice dynamics based on harmonic and quasi-harmonic level of theory. For simplicity, this tutorial is focused only on harmonic phonons. For quasi-harmonic part, please refer to 'thermodynamics-QHA'.

**Classes used in this tutorial are defined in 'CRYSTALpytools.thermodynamics'. Documentations and source codes are attached at the bottom of this page.**

## Tests
2 tests are performed. One is based on $\Gamma$ point calculations and the other one is based on phonon dispersion calculations.

1. Thermodynamic properties at room temperature (298.15K), 0 pressure - For comparison with CRYSTAL17 output.  
2. Thermodynamic properties from 0K to 300K, with the interval of 20K and from 0 GPa to 0.5 GPa with the interval of - To illustrate the structure of output file and key information. 

**N.B.** It is somehow meanless to consider pressure & Gibbs free energy at Harmonic level, since the volume is assumed as constant. $G(T, p)=F(T)+pV$.

### Room temperature (298.15K), 0 pressure thermodyanmics
This part is used for comparison with CRYSTAL17 output file, in order to illustrate the equivalence of methods adopted here and the method of CRYSTAL17. $\Gamma$ point calculation of Form I paracetamol is used ('freqf1-r0.out' in 'data/'). Reference data with the same environmental conditions can be found in the same file. No output file is generated. 

The 'pressure' specified is the value given in the same output file. For comparison only.

In [1]:
from CRYSTALpytools import thermodynamics
from CRYSTALpytools.thermodynamics import Harmonic

rm_condition = Harmonic(write_out=False).from_file(
    'data/freqf1-r0.out', scelphono=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], auto_calc=False
)
E_h, E_g, E_zp, U_vib, S, C_v = rm_condition.thermodynamics(
    temperature=[298.15], pressure=[0.10132500E-3]
)

# Note: All attributes are lists.
print('DFT total energy (EL) = ', rm_condition.edft, 'KJ/mol')
print('Helmholtz free energy (EL+E0+ET-TS) = ', E_h[0, 0], ' KJ/mol')
print('Gibbs free energy (EL+E0+ET+pV-TS) = ', E_g[0, 0, 0], ' KJ/mol')
print('Zero-point energy (E0) = ', E_zp[0], ' KJ/mol')
print('Vibrational contribution to interla energy - E0 (ET) = ', U_vib[0, 0] - E_zp[0], ' KJ/mol')
print('Entropy*Temperature (TS) = ', S[0, 0] * 298.15, ' J/mol')
print('Heat capacity = ', C_v[0, 0], ' J/mol*K')
print('Entropy = ', S[0, 0], ' J/mol*K')

NameError: name 'H_to_kjmol' is not defined

References from 'freqf1-r0.out' (line 11703~11738) are attached below. Here are several findings:

1. Parameters derived from vibrational frequencies shows good agreement with reference data, with the error at the level of 10^-4 KJ/mol. The choice of physical constants and different unit conversion coefficients might lead to the discrepancy, where the physical constants play a major role (See below).  
2. The difference in DFT total energy lies in different coefficients for unit conversion. In CRYSTAL17 case, 1 Hartree = 2625.5 KJ/mol. In the current case, 1 Hartree = 2625.500256 KJ/mol.  
3. The PV term is neglected at room condition. Helmholtz free energy is the focus of this tutorial since the Harmonic class was initially developed for QHA fittings. To calculate Gibbs free energy at HA level, use this equation: $G(T,p) = F(T) + pV$

```
 *******************************************************************************

 HARMONIC VIBRATIONAL CONTRIBUTIONS TO THERMODYNAMIC FUNCTIONS AT GIVEN
 TEMPERATURE AND PRESSURE:

 (EL = ELECTRONIC ENERGY
  E0 = ZERO-POINT ENERGY
  ET = THERMAL CONTRIBUTION TO THE VIBRATIONAL ENERGY
  PV = PRESSURE * VOLUME
  TS = TEMPERATURE * ENTROPY)

                          AU/CELL             EV/CELL                 KJ/MOL
 EL            :   -2057.686915559598  -55992.507576455653    -5402456.23545757
 E0            :       0.670005954735      18.231788914567        1759.10038625


 *******************************************************************************

 THERMODYNAMIC FUNCTIONS WITH VIBRATIONAL CONTRIBUTIONS

 AT (T =  298.15 K, P =   0.10132500E+00 MPA):

                          AU/CELL             EV/CELL                 KJ/MOL
 ET            :       0.040831798552       1.111089725297         107.20387199
 PV            :       0.000017530544       0.000477030356           0.04602644
 TS            :       0.080307302905       2.185272809177         210.84679406
 ET+PV-TS      :      -0.039457973810      -1.073706053524        -103.59689564
 EL+E0+ET+PV-TS:   -2057.056367578673  -55975.349493594607    -5400800.73196695

 OTHER THERMODYNAMIC FUNCTIONS:

                      mHARTREE/(CELL*K)     mEV/(CELL*K)              J/(MOL*K)
 ENTROPY       :       0.269352013769       7.329440916241         707.18361249
 HEAT CAPACITY :       0.253504784422       6.898215882635         665.57671770

 *******************************************************************************
```

### Multiple q points, Multiple temperatures & pressures
Class Harmonic can calculate the HA thermodynamics at a set of q points in order to sample various cell sizes. As a test case illustrating the effectiveness of this, a phonon dispersion calculation is performed based on the $\Gamma$ point frequency data of Form I paracetamol crystal. `SCELPHONO` is set to be 1 to save computation time.

The reference data is 'f1-disp.out' - Frequencies at $\Gamma$ are different from the 'f1freq-r0.out' case due to unknown reasons. In this case, nqpoint = 140, nmode = 240, natom = 80. The temperature range is set to be 0K ~ 300K, with intervals of 30K while the pressure range is set to be 0 GPa ~ 0.5 GPa, with intervals of 0.1 GPa. Since negative frequencies presents, warning messages will be printed. 

All the calculations are automatically executed. The output file is 'f1-disp-HA.dat'. Phonon dispersions are summed up, which, in the current case, is not necessary since the primitive cell is sufficiently large. This option is useful for smaller primitive cells.

In [ ]:
import numpy as np

t = np.arange(0, 301, 30)
p = np.arange(0, 0.51, 0.1)
Harmonic(
    temperature=t, pressure=p, write_out=True, filename='data/f1-disp-HA.dat'
).from_file(
    'data/f1-disp.out', scelphono=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
)